# Going Down the EECS Stack Lab: Security

-----

Welcome to EECS Decal Session 4: Computer Security!

In this demo, you will be exploring a few examples of common web attacks
through a sample to-do list application. Here is the web attacks that you will go over:

- XSS (Cross-Site-Scripting)
- SQL Injection
- Clickjacking

First, we will observe the effects of Cross-site scripting in two forms:

1. Stored XSS
2. Reflected XSS

Then, we will show an example of SQL Injection which may result in some
interesting effects to your to-do list website. ;)

# Common Web Attacks

## XSS (Cross-Site-Scripting) attacks

XSS attacks happens when an application receives and run data that contains malicious Javascript code. This can be done by passing input that contains a [`<script>` tag](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/script).

A `<script>` tag is a type of HTML (code that displays webpages) tag that lets you put Javascript (a programming language used on the web) in it. The script inside the script tag is executed as the browser processes the markup. 

For example: `<script>alert("lol")</script>`

Later in this lab, we will explore an example where we pass an input containing a `<script>` tag to our TODO list database and observe its payload when we reload the database. 

For more information about XSS attacks, see [here](https://www.acunetix.com/websitesecurity/cross-site-scripting/).

## SQL Injection

[SQL injection](https://owasp.org/www-community/attacks/SQL_Injection) is a code injection technique that allows an attacker to interfere the queries between an application and its database. Bad things can happen when unsanitized input is executed. For example, consider the following code that is part of a database system:

In [ ]:
#...
#... cursor = sqlite3.connect('db.sql3').cursor  # do not uncomment this
#... 
#...

happy_query = 'EECS Decal'

query = {'name': [happy_query]}
todo_item_name = query['name'][0]
sql = f"DELETE FROM todos WHERE items = '{todo_item_name}'"

#... cursor.executescript(sql)
print(sql)

The sql that the cursor will execute is `DELETE FROM todos WHERE items = 'EECS Decal'`.
The query will delete any item from our TODO list that is equal to our target phrase.
So far so good.
However, if an attacker inputs the following text instead:

In [ ]:
#...
#... cursor = sqlite3.connect('db.sql3').cursor  # do not uncomment this
#... 
#...

evil_query = "' OR ''='"

query = {'name': [evil_query]}
todo_item_name = query['name'][0]
sql = f"DELETE FROM todos WHERE items = '{todo_item_name}'"

#... cursor.executescript(sql)
print(sql)

This is not good. Since `''=''` is always true, the `DELETE` statement is going to delete every entry in the table.
But why stop at `DELETE`? Let's see what we can do with the `INSERT` command.
This is what we expect an `INSERT` command to do:

In [ ]:
#...
#... cursor = sqlite3.connect('db.sql3').cursor  # do not uncomment this
#... 
#...

happy_query = 'EECS Decal'

query = {'name': [happy_query]}
todo_item_name = query['name'][0]
sql = f"INSERT INTO todos VALUES('{todo_item_name}')"

#... cursor.executescript(sql)
print(sql)

If someone is not satisfied with deleting everything on the table, but wants to get rid of the table as well, they might come up with a mischievous command that looks like this:

In [ ]:
#...
#... cursor = sqlite3.connect('db.sql3').cursor  # do not uncomment this
#... 
#...

mischievous_query = "have a great day'); DROP TABLE todos--"

query = {'name': [mischievous_query]}
todo_item_name = query['name'][0]
sql = f"INSERT INTO todos VALUES('{todo_item_name}')"

#... cursor.executescript(sql)
print(sql)

This works since multiple SQL queries can be placed on the same line when separated by a semicolon.
Here's a comic from [xkcd](https://xkcd.com/327/) to explain what would happen in theory:

![](files/exploits_of_a_mom.png)

But fortunately, the server later in this demo does not respond to this attack. Phew.
Here's a blank page that you can experiment with to see what other malicious queries you can generate.

In [ ]:

student_query = ""

query = {'name': [student_query]}
todo_item_name = query['name'][0]
sql = f"{todo_item_name}"

## Clickjacking

Clickjacking is a technique that tricks the user into clicking something that is different from what the user sees ([wikipedia: Clickjacking](https://www.google.com/)).

<a style="color: inherit; text-decoration: none;" href="https://en.wikipedia.org/wiki/Clickjacking" title="No decoration">lol</a>. Here's the real link: [real link to wikipedia](https://www.google.com/)

... not really. The "lol" is the real real link if you click on it. 
I hope you get the point. What you see might not be what you think it is.
For this example, you can use the link preview function in your browser to avoid being tricked. However below you will see an example where even that fails.

# Demo (OPTIONAL)

First, run the following cell. If you are doing this on JupyterHub, do _not_ go to the link that is printed out, but follow the directions below.

In [ ]:
%run server.py

## Local Setup

If you already have Git and Jupyter Notebook installed, follow these instructions:

```bash
git clone https://github.com/HKNDecal/HKNDecal.git  # clone the repo
cd session4-security  # enter the security notebook folder
jupyter notebook  # spin up the Jupyter notebook
```

If you wish to use Python instead of Jupyter Notebook, you can replace the command `jupyter notebook` above with `python server.py`. This will directly run the website instead.

Here are the four demos that we will go over:

1. **Cross-Site-Scripting Vulnerability**:

  In the text field, copy/paste the following line and add the item:
  
```html
<script>alert("Alert! This is an XSS vulnerability")</script>
```
  Now, reload the page and see what happens.

2. **Relfected XSS Vulnerability**:

  Copy/paste the URL below in and refresh to see the effects:
	<http://localhost:1050/?person=%3Cimg%20%20src=%22broken%20link%22%20onerror=%22alert(%27Triggered%27)%22/%3E>

3. **SQL Injection**:

  In the text field, copy/paste the following line:
  
```sql
itemName'%3BDELETE FROM todos%3B--
```
  Add the item to the page, then refresh and examine behavior.
  Now try to delete the same text field.
  Do you notice something wrong?

4. **Clickjacking**:

  Click the link close to the bottom of the page and see what happens. ;)
  You may need to try clicking a few times.